# Frictionless Pendulum on a Cart

## Imports

In [27]:
import numpy as np
import sympy as sp

## Symbols

In [28]:
t, g = sp.symbols("t g")
l, m, M = sp.symbols("l m M")

theta = sp.Function("theta")
x = sp.Function("x")

# Phisical Modeling

## Kinematics

In [29]:
x_p = x(t) + l * sp.sin(theta(t))
x_p

l*sin(theta(t)) + x(t)

In [30]:
y_p = -l * sp.cos(theta(t))
y_p

-l*cos(theta(t))

In [31]:
x_p_dot = x_p.diff(t)
x_p_dot

l*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t)

In [32]:
y_p_dot = y_p.diff(t)
y_p_dot

l*sin(theta(t))*Derivative(theta(t), t)

## Potential Energy

There's only the gravitational potential energy, which acts in the `y` direction.

$$
V = m \cdot g \cdot y = - m \cdot g \cdot l \cdot cos(\theta)
$$

In [33]:
V = - m * g * l * sp.cos(theta(t))
V

-g*l*m*cos(theta(t))

## Kinectic Energy

There are two Kinectic energies, one related to the Cart and one on the Pendulum mass. The total energy is the sum of both.

Since the pendulum moves in two coordinates, it can be modeled as two separate kinectic energies.

$$
T = \frac{1}{2} \cdot M \cdot \dot{x}^2 + \frac{1}{2} \cdot m \cdot \dot{x}_p^2 + \frac{1}{2} \cdot m \cdot \dot{y}_p^2
$$

$$
T = \frac{1}{2} \cdot M \cdot \dot{x}^2 + \frac{1}{2} \cdot m \cdot (\dot{x}_p^2 + \dot{y}_p^2)
$$

In [34]:
T = sp.Rational(1, 2) * M * (x(t).diff(t) ** 2) + sp.Rational(1, 2) * m * (x_p_dot ** 2 + y_p_dot ** 2)
T.simplify()

M*Derivative(x(t), t)**2/2 + m*(l**2*Derivative(theta(t), t)**2 + 2*l*cos(theta(t))*Derivative(theta(t), t)*Derivative(x(t), t) + Derivative(x(t), t)**2)/2

## Lagrangian and Equations of Motion

In [35]:
L = T - V
L

M*Derivative(x(t), t)**2/2 + g*l*m*cos(theta(t)) + m*(l**2*sin(theta(t))**2*Derivative(theta(t), t)**2 + (l*cos(theta(t))*Derivative(theta(t), t) + Derivative(x(t), t))**2)/2

In [36]:
x_motion_eq, theta_motion_eq = sp.euler_equations(L, [x(t), theta(t)], t)

In [37]:
x_motion_eq

Eq(-M*Derivative(x(t), (t, 2)) - m*(-l*sin(theta(t))*Derivative(theta(t), t)**2 + l*cos(theta(t))*Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2))), 0)

In [38]:
theta_motion_eq.simplify()

Eq(l*m*(g*sin(theta(t)) + l*Derivative(theta(t), (t, 2)) + cos(theta(t))*Derivative(x(t), (t, 2))), 0)

# Coupled Equations

In the previous section, both Euler-Lagrange equations of motion are coupled to each other's second order derivative:

$$
\ddot{x} = f(t, x, \dot{x}, \ddot{x}, \theta, \dot{\theta}, \ddot{\theta}) \\
\ddot{\theta} = f(t, x, \dot{x}, \ddot{x}, \theta, \dot{\theta}, \ddot{\theta})
$$

Before linearizing them, It's necessary to solve the system so that it's possible to write $\ddot{x}$ and $\ddot{\theta}$ as only functions of lower order derivatives:

$$
\ddot{x} = f(t, x, \dot{x}, \theta, \dot{\theta}) \\
\ddot{\theta} = f(t, x, \dot{x}, \theta, \dot{\theta})
$$


In [39]:
sols = sp.solve([x_motion_eq, theta_motion_eq], [x(t).diff(t, 2), theta(t).diff(t, 2)])

In [40]:
x_ddot = sols[x(t).diff(t, 2)].simplify()
x_ddot

m*(g*cos(theta(t)) + l*Derivative(theta(t), t)**2)*sin(theta(t))/(M + m*sin(theta(t))**2)

In [41]:
theta_ddot = sols[theta(t).diff(t, 2)].simplify()
theta_ddot

-(M*g + g*m + l*m*cos(theta(t))*Derivative(theta(t), t)**2)*sin(theta(t))/(l*(M + m*sin(theta(t))**2))

# Linearization

From the previous section, it's possible to write the system of equations in this format:

$$
\frac{d}{dt} \begin{bmatrix}
  x \\
  \dot{x} \\
  \theta \\
  \dot{\theta}
\end{bmatrix}
= \begin{bmatrix}
  f_1(t, x, \dot{x}, \theta, \dot{\theta}) \\
  f_2(t, x, \dot{x}, \theta, \dot{\theta}) \\
  f_3(t, x, \dot{x}, \theta, \dot{\theta}) \\
  f_4(t, x, \dot{x}, \theta, \dot{\theta})
\end{bmatrix}
= \begin{bmatrix}
  \dot{x} \\
  \ddot{x} \\ 
  \dot{\theta} \\
  \ddot{\theta}
\end{bmatrix}
$$

## Jacobian Matrix

The first step to linearize this sytems is to compute the Jacobian matrix:

$$
\frac{D_f}{D_x} = \begin{bmatrix}
    \frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial \dot{x}} & \frac{\partial f_1}{\partial \theta} & \frac{\partial f_1}{\partial \dot{\theta}} \\[0.5em]
    \frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial \dot{x}} & \frac{\partial f_2}{\partial \theta} & \frac{\partial f_2}{\partial \dot{\theta}} \\[0.5em]
    \frac{\partial f_3}{\partial x} & \frac{\partial f_3}{\partial \dot{x}} & \frac{\partial f_3}{\partial \theta} & \frac{\partial f_3}{\partial \dot{\theta}} \\[0.5em]
    \frac{\partial f_4}{\partial x} & \frac{\partial f_4}{\partial \dot{x}} & \frac{\partial f_4}{\partial \theta} & \frac{\partial f_4}{\partial \dot{\theta}}
\end{bmatrix}
$$

In [42]:
eq_system = sp.Matrix([
    x(t).diff(t),
    x_ddot,
    theta(t).diff(t),
    theta_ddot
])
eq_system


Matrix([
[                                                                                   Derivative(x(t), t)],
[             m*(g*cos(theta(t)) + l*Derivative(theta(t), t)**2)*sin(theta(t))/(M + m*sin(theta(t))**2)],
[                                                                               Derivative(theta(t), t)],
[-(M*g + g*m + l*m*cos(theta(t))*Derivative(theta(t), t)**2)*sin(theta(t))/(l*(M + m*sin(theta(t))**2))]])

In [43]:
vars = [x(t), x(t).diff(t), theta(t), theta(t).diff(t)]
jacobian = eq_system.jacobian(vars)
jacobian

Matrix([
[0, 1,                                                                                                                                                                                                                                                                                                              0,                                                                                 0],
[0, 0,                                                -g*m*sin(theta(t))**2/(M + m*sin(theta(t))**2) - 2*m**2*(g*cos(theta(t)) + l*Derivative(theta(t), t)**2)*sin(theta(t))**2*cos(theta(t))/(M + m*sin(theta(t))**2)**2 + m*(g*cos(theta(t)) + l*Derivative(theta(t), t)**2)*cos(theta(t))/(M + m*sin(theta(t))**2),              2*l*m*sin(theta(t))*Derivative(theta(t), t)/(M + m*sin(theta(t))**2)],
[0, 0,                                                                                                                                                                                                   

## Fixed Point

In [44]:
fixed_point = {x(t): 0, x(t).diff(t): 0, theta(t): sp.pi, theta(t).diff(t): 0}

In [45]:
jacobian.subs(fixed_point)

Matrix([
[0, 1,                 0, 0],
[0, 0,             g*m/M, 0],
[0, 0,                 0, 1],
[0, 0, (M*g + g*m)/(M*l), 0]])